### ETPOL: Encoder Transformer for Political Orientation Labeling
###### Machine Learning Final Project: Fynn Hayton-Ruffner

#### What do I want to do?
Of all the different application areas for machine learning discussed in this course, I have been most interested in text analysis, and, since it wasn't a major focus this semester, I thought it would be a fun task to focus on for my final project. Concisely, my hope for this project is to build a transformer that can accurately classify social media posts as liberal or conservative. Why is this interesting? For one, a considerable plurality of the things we say and the ideas that we espose are tied closely with our political beliefs, whether we are conscious of it or not. Political inclination may be evident not only in explict commentary on political events, but in everyday conversations as well. With this in mind, one could apply this model to any social media thread to gain an understanding of the dynamics of the discourse surrounding that topic (at least on that particular platform). Is there a group dominating the discussion, or is it more balanced? Application areas that immediately come to mind are threads focusing on climate change, the current admistration, or other overtly political topics, but the actual realm of possibility is pretty endless. As a really out-there example, you could look at a thread about the Kardashians and assess the political inclinations of those posting. Are conservatives or liberals more passionate about the Kardashians? On a high level, this task provides broad insights into who cares about what, and that interests me a lot as someone who cares about politics, while admittingly not being the most informed individual. If I create a good model (> 75-80% accuracy), I will apply it to twitter threads of interst using the twitter API. If I am successful in this, I'll include a report as part of my final submission to illustrate my findings. 

#### How am I doing it?

##### The Data
To start, I had to find a sizeable number of social media posts labeled by political inclination. Luckily, I found two datasets on kaggle that had just the thing: [Messy Twitter Dataset](https://www.kaggle.com/datasets/cascathegreat/politicaltweets) with ~130,000 posts, and [Clean Reddit Dataset](https://www.kaggle.com/datasets/neelgajare/liberals-vs-conservatives-on-reddit-13000-posts/data) with 13,000 posts. However, while I was lucky to find these datasets, they were structured completely differently, so in order to join them I had to do a lot of cleaning. I have the full cleaning process in 'clean_data.ipynb' but to sum up, I reformated the post content of the messy dataset (such a pain), removed everything that wasn't alphanumeric or punctuation with regex, renamed columns for consistency, and dealt with invisible unicode chars. When I hopefully use the resulting model on twitter threads, I will have to do apply this same cleaning process to that data for consistency. Nonetheless, after all that, I was able to join the two datasets together to create an 'all_posts.csv' file with 146,478 rows and two columns: 'content' & 'affiliation' (with values left or right):

In [1]:
import pandas as pd
from pathlib import Path

df=pd.read_csv(Path('data/all_posts.csv'))
df.head()

,content,affiliation
0,when you look at the history of big social mov...,left
1,it was great to be back in new jersey! there's...,left
2,"virginians delivered for me twice, and now im ...",left
3,some of the most important changes often start...,left
4,glad i had a chance to talk with our new champ...,left


##### Next Step: Tokenizer
After preprocessing the data, the next step in text analysis is to create a tokenizer that maps text to the numerical format necessary to pass it through a transformer. The full code is available in 'tokenizer.py' but essentially I created a word piece tokenizer following the tutorial on [huggingface.com](https://huggingface.co/learn/llm-course/en/chapter6/8). Word piece is a subword tokenizer that breaks words into meaningful sub-units. For example, 'modernization' may become 'modern' & 'ization' since 'ization' may be encountered as a common suffix in the vocabulary. This tackles the common issue of unknown words in new data. Since the tokenizer always starts from every character encountered in the vocab, as long as the input text doesn't have any new characters it will be able to handle it. Given the size of the dataset and that I formatted the input data to only include alphanumeric chars and punctuation, it is very unlikely for the tokenizer to encounter characters not already in its vocabularly. After training the tokenizer on my dataset in tokenizer.py, I saved it in the 'wp_tokenizer' directory. It has a 20,000 token vocabulary, which was more than enough to account at least for all the unique characters in my dataset.

##### The actual model
As mentioned above, I decided to try to code my own encoder model using the pytorch module, since transformers tend to outperform any other machine learning model on text related tasks by a wide margin. The reason for this is attention, the part of a transformer that allows every token in the input sequence to attend to all other tokens (or just the preceding ones in the case of a decoder) and update its meaning to more accurately reflect the truth. To create the model, I followed the structure of an encoder from the [Attention is All You Need](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) paper, which introduced transformers to the world. The component flow for any input sequence X is:

X shape: (512) -> 512 tokens in input sequence

1: embedding layer: this converts all tokens to an embedding vector of size 'd_model'. Each word or subword becomes a long sequence of torch.longs that encodes its initial meaning before attention and learning.

X shape: (512 x 512): now every token is represented by an embedding vector of dimension 512

2: Positional Encoder: I followed the math directly from the paper. The positional encoder is a context_size x d_model matrix that holds a vector containing embeddings for every possible position in the input sequence. When the input passes through here, the positional embeddings are added to the corresponding input embeddings with the hope of adding positional information to each token's represenation.

X shape: (512 x 512): still the same, each token embedding is just updated by the positional encoder.

3: Encoder block:

This layer holds the bulk of the work. The token embeddings are first split across multiple attention heads, each assigned to different sub components of a token's meaning, and then updated by the embeddings of all other tokens in the sequence. The embeddings that result from attention are then added back to the inital token embeddings and passed through a standard multilayer perceptron network. I'm leaving out layer normalization and dropout which are methods to help with training and reducing overfitting because the bulk of the job is done with the attention -> feedforward flow, encapsulating one encoder block. Transformers typically stack encoder blocks to add as much information to the original tokenized inputs as possible. I incorporate the number of encoder blocks as a hyperparameter to my model 'num_encoders'. 

X shape: (512 x 512), the token embeddings are now richly encoded by the context provided by their surrounding neighbors.

4: Linear output: after being passed through the linear output, I extract the [CLS] token, which is inserted at the beginning of every input sequence. This token is meant to contain the label for classfication tasks after sufficient training, and is a common method for classfication in transformers. I then extract the class probablities it assigns to liberal and conservative, and use that to update all the parameters of the model.

X shape (2): just raw logit probs for each class

Below is an example pass through the model using a random observation in the dataset.


In [2]:
import torch
import torch.nn as nn
from transformers import PreTrainedTokenizerFast
from models import ETPOL
from pathlib import Path
from PostsDataset import PostsDataset

context_length = 512
d_model = 512

model = ETPOL(
    vocab_size=20000,
    context_length=context_length,
    d_model=d_model,
    num_heads=8,
    num_hidden_layers=2,
    d_hidden=2048,
    num_encoders=8
)

tokenizer = PreTrainedTokenizerFast.from_pretrained(Path('wp_tokenizer'), model_max_length=context_length)
# creates a dataset object from the dataframe, which internally handles tokenizing the inputs
dataset = PostsDataset(df, tokenizer, context_length)
rand_example = dataset.__getitem__(3)





In [3]:
reverse_label_map = {1: 'left', 0: 'right'} # this is the scheme I used to encode labels
inputs = rand_example['input_ids'].unsqueeze(0) # add the batch dimension
label = rand_example['labels']
outputs = model(inputs)
print(outputs.shape) # class probs, one for each class, first dimension is batch size


torch.Size([1, 2])


Here you can see what the input was, its political affiliation, and what the model predicted. It got it right but with no training and two classes to choose from there is nothing of significance here.

In [4]:
print(f"input post was: {tokenizer.decode(rand_example['input_ids'])}")
print(f"its label was: {reverse_label_map[int(label)]}")
print(f"It was predicted to be: {reverse_label_map[int(torch.argmax(outputs, dim=-1))]}")

input post was: [CLS] some of the most important changes often start in state legislatures . that ' s why i ' m proud to support these candidates for the virginia state legislature . i hope you ' ll join me and the d ##lc ##c in giving them your vote . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

##### Status

As of right now, I have my model code, a training loop that saves the model weights and its performance on accuracy, f1, precision, recall, and loss with a certain configuration of hyperparameters to a txt file (see results folder for a test run on 32 examples). I now just have connect to the hpc to train it on the full dataset and test out different configurations. Finally, if I get a successful model (75-80+% accuracy) I will apply it to interesting twitter threads.

##### Code

I'm attaching my whole directory of code here, so you can run any file in the root (don't try running old files in the tests folder). You will have to install various dependencies like torch, transformers, and evaluate in order to do this.